In [ ]:
import requests
import time
from pwn import *
import monocypher
import crypto
import json

In [ ]:
# big ugly utility class that i didn't know what to call
class Magic:
    def __init__(self):
        self.SERVER_URL = None
        self.shared_key1 = None
        self.shared_key2 = None

    def receive_radio_messages(self, p=False):
        time.sleep(0.1)
        messages = requests.get(self.SERVER_URL+"/radio_rx").json()

        if p:
            print()
            for msg in messages:
                print("DEBUG: Received message", msg)

        return messages

    def receive_next(self):
        while not (msg := self.receive_radio_messages()): pass
        return msg[0] # in hindsight i should have made a read buffer ¯\_(ツ)_/¯

    def inject_radio_message(self, message):
        requests.post(self.SERVER_URL+"/radio_tx", json=message)

    def start_robot(self):
        requests.get(self.SERVER_URL+"/start")

    def stop_robot(self):
        requests.get(self.SERVER_URL+"/stop")

    def get_board_state(self):
        return requests.get(self.SERVER_URL+"/state").json()

    def decrypt(self, enc, key):
        return json.loads(crypto.decrypt(enc, key).decode())

    def encrypt(self, obj, key):
        return crypto.encrypt(json.dumps(obj), key)

    def convert(self, enc, key1, key2):
        return self.encrypt(self.decrypt(enc, key1), key2)

    def receive_next_robot(self, t=False):
        msg = self.receive_next()
        if t:
            return self.decrypt(msg['encrypted'], self.shared_key1), msg["msg_type"]
        return self.decrypt(msg['encrypted'], self.shared_key1)

    def receive_next_controller(self, t=False):
        msg = self.receive_next()
        if t:
            return self.decrypt(msg['encrypted'], self.shared_key2), msg["msg_type"]
        return self.decrypt(msg['encrypted'], self.shared_key2)

    def send_to_robot(self, msg_type, msg):
        payload = {
            'msg_type': msg_type,
            'src': 0x10,
            'dst': 0x20,
            'encrypted': self.encrypt(msg, self.shared_key1)
        }
        self.inject_radio_message(payload)

    def send_to_controller(self, msg_type, msg):
        payload = {
            'msg_type': msg_type,
            'src': 0x10,
            'dst': 0x30,
            'encrypted': self.encrypt(msg, self.shared_key2)
        }
        self.inject_radio_message(payload)

In [ ]:
def setup(magic): # solves challenge and gets DH keys
    input()
    magic.start_robot()

    # recv challenge
    challenge = magic.receive_next()

    # send challenge
    challenge['dst'] = 0x30; challenge['src'] = 0x10
    magic.inject_radio_message(challenge)

    # send results
    response = magic.receive_radio_messages()[0]
    response['dst'] = 0x20; challenge['src'] = 0x10
    magic.inject_radio_message(response)

    priv_key = b'A' * 32

    # compute key with robot
    key_exchange = magic.receive_radio_messages()[1]
    magic.inject_radio_message({"msg_type": "ack_key_exchange", "src": 0x10, "dst": 0x20, "key": monocypher.compute_key_exchange_public_key(priv_key).hex()})
    magic.shared_key1 = monocypher.key_exchange(priv_key, bytes.fromhex(key_exchange['key']))

    # save for later (this is a secure_data 'get_message' msg)
    msg = magic.receive_next_robot()

    # compute key with controller
    magic.inject_radio_message({"src": 0x10, "dst": 0x30, "msg_type": "key_exchange", "key": monocypher.compute_key_exchange_public_key(priv_key).hex()})
    key_exchange = magic.receive_radio_messages()[0]
    magic.shared_key2 = monocypher.key_exchange(priv_key, bytes.fromhex(key_exchange["key"]))

    return msg

In [ ]:
# 0x10: me
# 0x20: robot
# 0x30: controller

magic = Magic()
magic.SERVER_URL = "http://activist-birds.picoctf.net:57367/"

magic.inject_radio_message({'msg_type': 'set_addr', 'new_addr': 0x30, 'dst': 0x10})
magic.receive_radio_messages()

#######################################################
# RUN 1: collect normal data
#######################################################

print("run 1")
msg = setup(magic)
msg_type = "secure_data"

print("collecting data")

data = [] # collecting normal messages from nonce 

for i in range(4):
    if i > 0: msg, msg_type = magic.receive_next_robot(True)
    if i == 3: magic.stop_robot()

    magic.send_to_controller(msg_type, msg)
    res, res_type = magic.receive_next_controller(True)
    magic.send_to_robot(res_type, res)

    data.append(msg)
    data.append(res)

#######################################################
# RUN 2: get offset movements
#######################################################

print("run 2")
msg = setup(magic)

print("creating offset")
for i in (1, 2, 5, 6):
    magic.send_to_controller("secure_data_request", data[i])

magic.receive_radio_messages()

magic.send_to_robot("secure_data_ack", data[1])
magic.receive_next()
magic.send_to_robot("secure_data_response", data[3])
magic.receive_next()
magic.send_to_robot("secure_data_ack", data[5])
magic.receive_next()
magic.send_to_robot("secure_data_response", data[7])

print('collecting offsets')

offsets = []
for i in range(10):
    msg, msg_type = magic.receive_next_robot(True)
    magic.send_to_controller(msg_type, msg)

    if i == 9: magic.stop_robot()
    res, res_type = magic.receive_next_controller(True)
    magic.send_to_robot(res_type, res)

    if i % 4 == 1: # gives E5, W9, E13
        offsets.append(res)


#######################################################
# RUN 2: hacking time!!!
#######################################################

print("run 3")
msg = setup(magic)
msg_type = "secure_data"

print("hacking!")

for i in range(20):
    if i > 0: msg, msg_type = magic.receive_next_robot(True)
    if i == 19: magic.stop_robot()

    magic.send_to_controller(msg_type, msg)
    res, res_type = magic.receive_next_controller(True)

    if i == 5: res = offsets[0]
    if i == 9: res = offsets[1]
    if i == 13: res = offsets[2]
    magic.send_to_robot(res_type, res)